In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import urllib.request, time, os

In [2]:
def ts(x):
    time.sleep(x)

## 1. 

In [3]:
# //*[@id="query"]
# //*[@id="place-main-section-root"]/section/div/div[6]/a

In [20]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())); ts(2)
driver.get('https://www.naver.com'); ts(2)

driver.find_element(By.ID, 'query').send_keys('관악구 돈가스' + '\n'); ts(3)

driver.find_element(By.XPATH, '//*[@id="place-main-section-root"]/section/div/div[6]/a').click()

driver.switch_to.window(driver.window_handles[1])


# iframe 안으로 들어가기
driver.switch_to.frame('searchIframe')

# iframe 안쪽을 한번 클릭하기
driver.find_element(By.ID, '_pcmap_list_scroll_container').click()

# 로딩된 데이터 개수 확인
res = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
before_len = len(res)

# 페이지 아래로 내리기
while True:
    # 맨 아래로 스크롤 내린다.
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)

    # 스크롤 사이 페이지 로딩 시간
    ts(2)

    # 스크롤 후 로딩된 데이터 개수 확인
    res = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
    after_len = len(res)

    # 로딩된 데이터 개수가 같다면 반복 멈춤
    if before_len == after_len:
        break
    before_len = after_len

# 첫 번째 음식점부터 클릭
driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li:nth-child(1)').click()
ts(1)

In [22]:
# iFrame 나가 기본 콘텐츠로 돌아가기
driver.switch_to.default_content()

# iframe 전환하기
driver.switch_to.frame('entryIframe') # 상세정보

ElementClickInterceptedException: Message: element click intercepted: Element <div id="_title" class="YouOG">...</div> is not clickable at point (192, 13). Other element would receive the click: <h1 id="_header" class="bh9OH" style="opacity: 1;">...</h1>
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002B2B63+47795]
	(No symbol) [0x002467E1]
	(No symbol) [0x0014C7ED]
	(No symbol) [0x0017DF96]
	(No symbol) [0x0017CC09]
	(No symbol) [0x0017B3DB]
	(No symbol) [0x0017A835]
	(No symbol) [0x00173130]
	(No symbol) [0x00194CCC]
	(No symbol) [0x00172BC6]
	(No symbol) [0x00195024]
	(No symbol) [0x001A71DA]
	(No symbol) [0x00194AC6]
	(No symbol) [0x001713D7]
	(No symbol) [0x0017255D]
	GetHandleVerifier [0x00506F89+2490073]
	GetHandleVerifier [0x0054C361+2773681]
	GetHandleVerifier [0x00546211+2748769]
	GetHandleVerifier [0x003316B0+566784]
	(No symbol) [0x0025019A]
	(No symbol) [0x0024C418]
	(No symbol) [0x0024C4FB]
	(No symbol) [0x0023F317]
	BaseThreadInitTh

In [4]:

res_name = driver.find_element(By.CSS_SELECTOR, '#_title > span.Fc1rA').text
res_cate = driver.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text
res_score = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
res_review_cnt = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em').text
res_blog_cnt = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em').text
res_address = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[1]/div/a/span[1]').text
res_facility = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div/div[2]/div/div/div[6]/div').text
res_broad = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div/div[2]/div/div/div[7]/div/a/div[1]/div/span[1]').text
res_info = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div/div[2]/div/div/div[9]/div/a/span[1]').text





# //*[@id="_title"]/span[1]

# #app-root > div > div > div

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="_title"]"}
  (Session info: chrome=115.0.5790.171); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C82B63+47795]
	(No symbol) [0x00C167E1]
	(No symbol) [0x00B1C7ED]
	(No symbol) [0x00B49AC8]
	(No symbol) [0x00B49B8B]
	(No symbol) [0x00B78A82]
	(No symbol) [0x00B64D14]
	(No symbol) [0x00B771DA]
	(No symbol) [0x00B64AC6]
	(No symbol) [0x00B413D7]
	(No symbol) [0x00B4255D]
	GetHandleVerifier [0x00ED6F89+2490073]
	GetHandleVerifier [0x00F1C361+2773681]
	GetHandleVerifier [0x00F16211+2748769]
	GetHandleVerifier [0x00D016B0+566784]
	(No symbol) [0x00C2019A]
	(No symbol) [0x00C1C418]
	(No symbol) [0x00C1C4FB]
	(No symbol) [0x00C0F317]
	BaseThreadInitThunk [0x75D000C9+25]
	RtlGetAppContainerNamedObjectPath [0x77EA7B1E+286]
	RtlGetAppCo

In [ ]:
# #_pcmap_list_scroll_container > ul > li:nth-child(1)
# #_pcmap_list_scroll_container > ul > li:nth-child(50)

# #_pcmap_list_scroll_container > ul > li:nth-child(50)

In [19]:
soup.select('iframe')

[]

In [5]:
after_len

50

In [5]:
driver.window_handles

['82FCFEFD43638380FCBB4011D298D985', 'DE5E1DAC4932DA21BBA20C8351C26695']

In [ ]:
driver.switch_to.frame()

In [30]:
driver.close(); ts(1)
driver.switch_to.window(driver.window_handles[0])
driver.close()
driver.quit()